# Navier Stokes Equation Reduced Basis Method 
Incompressible timedependet Navier Stokes equation

Find $(u,p):[0,T] \to (H_{0,D}^1)^d \times L^2$, s.t.

\begin{align}
\int_{\Omega} \partial_t u \cdot v + \int_{\Omega} \nu \nabla u \nabla v + u \cdot \nabla u v - \int_{\Omega} \operatorname{div}(v) p &= \int f v  && \forall v \in (H_{0,D}^1)^d, \\ 
- \int_{\Omega} \operatorname{div}(u) q &= 0 && \forall q \in L^2, \\
\quad u(t=0) & = u_0
\end{align}

In [1]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [2]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)
# viscosity
nu = 0.001
dt = 0.001

T = 1
dn = int(T/dt)
z = 100  # Number of snapshots

In [3]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [4]:
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu.components[0],mesh,"gfu",sd=3)

In [5]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res

In [6]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()

mh = BilinearForm(X)

mh += InnerProduct(u,v)*dx
mh.Assemble()

inv = mstar.mat.Inverse(X.FreeDofs())


conv = LinearForm(X)
conv += InnerProduct(grad(velocity)*velocity,v)*dx

In [7]:
rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Fh = f.vec
rows,cols,vals = mh.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))

In [8]:
t = 0
tend = 0
erg_h = np.zeros([len(gfu.vec),int(dn)+1])
conv_h = np.zeros([len(gfu.vec),int(dn)])
V = np.zeros([len(gfu.vec),z]) # creat the orthogonal transformationsmatrix for the reducebasis method
line = np.linspace(0,dn-1,z) # stors the number of linear distributed number of iteration from the solution
S_POD = np.zeros([len(gfu.vec),z])
#line = np.linspace(0,z,z) # stors the number of linear distributed number of iteration from the solution
#line

$
M\bigg(\frac{u^{n+1}-u^n}{k}\bigg) + A u^{n+1} +C u^{n} = 0
$

$
M^*\bigg(u^{n+1}-u^n\bigg) = -k A u^n - k Cu^n
$

$
M^* = M + kA 
$

In [9]:
# implicit Euler/explicit Euler splitting method:
erg_h[:,0] = gfu.vec
S_POD[:,0] = gfu.vec
o = 1
e = int(line[o])
for i in range(int(dn)):
    conv.Assemble()
    res.data = a.mat * gfu.vec + conv.vec
    gfu.vec.data -= dt * inv * res
    
    for r in line:
        if int(r) == i: #desided which solution will be orthogonalirsed
            if i > 0:
                S_POD[:,o] = gfu.vec
                o = o + 1
    erg_h[:,i+1] = gfu.vec #stores the solution for comparing later
    t = t + dt
    Redraw()

## POD method
A other way to get the transforamtions matrix is the POD method.
gifen $\mathbb{F} = \{u_h^0,u_h^1,...u_h^{S-1}\}$ where $S$ ist the number of snapshots.

$
C = \mathbb{F}^T\mathbb{F}\\
$

$
C \in \mathbb{R}^{S \times S}
$

and solve the eigenwalueproblem

$
C\Omega = \Omega\Lambda\\
\Lambda = diag[\lambda_0,...,\lambda_{S-1}], \Omega = [\omega^0,...,\omega^{S-1}]
$

$\lambda_i$ is the $i$th eigenvalue and $\omega^i$ is the corresponding normalized $i$th eigenvector

For a integer $r<<S$ given the $r$th POD basis like:

$
\phi_j = \frac{1}{\sqrt{\lambda_j}}\sum_{n=0}^{r-1}\omega^j_nu_h^n, \,\,\,\,\,\,\,\, j = 0,...,r-1
$

Frage:
For $j=0$  and $r = 3$ the expression becomes to:

$
\phi_0 = \frac{1}{\sqrt{\lambda_0}}\bigg[\omega_0^0 \cdot u_h^0 + \omega_1^0 \cdot u_h^1 +\omega_2^0 \cdot u_h^2\bigg]
$

So $\phi \in \mathbb{R}^{r\times N_h}$

To get a orthogonal basis $V = \{\phi_0,...\phi_{r-1}\}$

In [ ]:
def POD(S,r):
    #V = np.zeros([len(gfu.vec),r])
    ubar = np.zeros(r)
    #splitting the snapshotsolution like in the paper
    for i in range(S.shape[1]):
        mean = 0
        for j in range(S.shape[1]):
            mean += S[j,i]
        mean = 1/S.shape[1] * mean  
        
        if i < r:
            ubar[i] = mean
                        
        S[:,i] = S[:,i] - mean


    
    if S.shape[1] <= S.shape[0]:
        C = np.transpose(S).dot(S)
        U, Sig, VH = np.linalg.svd(C, full_matrices=True)
        V = U[:,0:r]
        #lam,omg = np.linalg.eigh(C)
        #for j in range(r):
        #    for i in range(r):
        #        V[:,j] += omg[i,j]*S[:,j]
        #    V[:,j] = 1/np.sqrt(lam[i]) * V[:,j]
        
        
        
        #for i in range(r):
        #    V[:,i] = 1/np.sqrt(lam[i]) * S.dot(omg[:,i])
        #V = V[0:r,0:r]
        #S = S[0:r,0:r]
        #for i in range(r):
        #    for j in range(r):
        #        V[:,i] += omg[j,i]*S[:,j]
        #    V[:,i] = 1/np.sqrt(lam[i]) * V[:,i]
        
        #U, Sig, VH = np.linalg.svd(C, full_matrices=True)
        #VH = np.transpose(VH)
        #S_N = S[0:r,0:r]
        #print(S_N)
        
        #for j in range(r):
        #    for i in range(r):
        #        V[:,j] += VH[i,j]*S[:,i]
        #    V[:,j] = 1/np.sqrt(Sig[j]) * V[:,j]
            
        #for i in range(S_N.shape[1]):
        #    V[:,i] = 1/np.sqrt(Sig[i]) * S_N.dot(VH_N[:,i])
        
        
    return V,ubar#,Sig,ubar

In [ ]:
r = 6
#V,Sig,ubar = POD_pap(S_POD,r)
V,ubar = POD(S_POD,r)
#print(V[:,0].dot(V[:,0]))
ubar

In [ ]:
print(V[:,0].dot(V[:,0]))
print(V.shape)

# Offline decomposition

$C_h(\omega,u,v) = \int_K(\omega \otimes u): \nabla v$

$C0_j = C_h(\bar{u}_h,\bar{u}_h,\phi_j)$

$\bar{u}_h = \frac{1}{S}\sum_{n=0}^{S-1}u_h^n$ is the mean walue 

In [ ]:
def Ch(wh,uh,vh):
    return np.linalg.norm(np.inner(np.outer(wh,uh),np.gradient(vh)))
    

In [ ]:
C0 = np.zeros(r)
w = ubar
u = w 
for i in range(r):
    v = V[:,i]
    #print(v)
    #print(np.gradient(v))
    #C0[i] = np.linalg.norm(np.inner(np.outer(w,u),np.gradient(v)))
#w = 
#v = np.array([1,0,-2,2,1,3])
#w = np.array([[3],[0],[-1],[-2],[1],[-3]])
#u = np.array([[-3],[5],[-1],[-2],[-1],[-3]])
#np.inner(np.outer(w,u),np.gradient(v))
#np.gradient(v)

In [ ]:
C = np.zeros([r,r,r])
for i in range(r):
    w = V[:,i]
    for j in range(r):
        u = V[:,j]
        for k in range(r):
            v = V[:,k]
            C[i,j,k] = Ch(w,u,v)
            
C1 = np.zeros([r,r])
for i in range(r):
    w = ubar[i]
    u = V[:,i]
    for j in range(r):
        v = V[:,j]
        C[i,j] = Ch(w,u,v)
        C[i,j] += Ch(u,w,v)
    

In [ ]:
C[0,:,:]

In [ ]:
C[1,:,:]